In [1]:
import keras
import matplotlib.pyplot as plt
import numpy as np

from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
model1 = load_model('1024_512_dl_2dropout_1conv_100ep.h5')

In [4]:
model1.summary()

test_dir = '/home/apando/datasets/building_crops_noaa_rezoom_des/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary')
test_results = model1.evaluate_generator(test_generator, steps=len(test_generator))
test_results

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
__________

[0.41173605993390083, 0.8333333333333334]

In [16]:
# F1 score on Maria small/clean test dataset, model as-is.
from sklearn import metrics

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary', shuffle=False)

predictions = model1.predict_generator(test_generator, steps=len(test_generator))

val_targ = []
for i in range(0,len(test_generator)):
    val_targ.extend(np.array(test_generator[i][1]))

val_predict = (np.asarray(predictions)).round().flatten()

print(val_targ[50:100])
print(val_predict[50:80])

val_f1 = metrics.f1_score(val_targ, val_predict)
val_recall = metrics.recall_score(val_targ, val_predict)
val_precision = metrics.precision_score(val_targ, val_predict)
val_acc = metrics.accuracy_score(val_targ, val_predict)

print("F1: {0}, Recall: {1}, Precision: {2}, Accuracy: {3}".format(val_f1, val_recall, val_precision, val_acc))

Found 384 images belonging to 2 classes.
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 1. 0. 0. 1. 1. 0.]
F1: 0.8626609442060087, Recall: 0.8933333333333333, Precision: 0.8340248962655602, Accuracy: 0.8333333333333334


In [12]:
# Misclassified examples
from keras.preprocessing import image

y_pred = []
y_label = []
for i in range(5):
    x,y = test_generator.next()
    y = y.ravel()
    temp_y = model1.predict(x)
    temp_y = np.round(temp_y.ravel())
    misclassification = np.absolute(y-temp_y)

    #show the misclassification
    misclass_index = np.where(misclassification > 0.5)
    for j in range(len(misclass_index[0])):
        plt.figure()
        #print(misclass_index[0][j])
        plt.title('label is {}, prediction is {}'.format(y[misclass_index[0][j]],temp_y[misclass_index[0][j]]))
        plt.imshow(image.array_to_img(x[misclass_index[0][j]]))
    misclass_index = None

NameError: name 'model1' is not defined

In [5]:
model2 = load_model('1024_512_dl_2dropout_1conv_100ep_balanced.h5')

In [6]:
model2.summary()

test_dir = '/home/apando/datasets/building_crops_noaa_rezoom_des/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary')
test_results = model2.evaluate_generator(test_generator, steps=len(test_generator))
test_results

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
__________

[0.4107826203107834, 0.8255208333333334]

In [13]:
# F1 score on Maria small/clean test dataset, model as-is.
from sklearn import metrics

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary', shuffle=False)

predictions = model2.predict_generator(test_generator, steps=len(test_generator))

val_targ = []
for i in range(0,len(test_generator)):
    val_targ.extend(np.array(test_generator[i][1]))

val_predict = (np.asarray(predictions)).round().flatten()

print(val_targ[50:100])
print(val_predict[50:80])

val_f1 = metrics.f1_score(val_targ, val_predict)
val_recall = metrics.recall_score(val_targ, val_predict)
val_precision = metrics.precision_score(val_targ, val_predict)
val_acc = metrics.accuracy_score(val_targ, val_predict)

print("F1: {0}, Recall: {1}, Precision: {2}, Accuracy: {3}".format(val_f1, val_recall, val_precision, val_acc))

Found 384 images belonging to 2 classes.
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 1. 1. 0.]
F1: 0.8571428571428571, Recall: 0.8933333333333333, Precision: 0.8237704918032787, Accuracy: 0.8255208333333334


In [7]:
model3 = load_model('1024_512_dl_3dropout_1conv_100ep.h5')

In [8]:
model3.summary()

test_dir = '/home/apando/datasets/building_crops_noaa_rezoom_des/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary')
test_results = model3.evaluate_generator(test_generator, steps=len(test_generator))
test_results

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
__________

[0.4291032813489437, 0.8046875]

In [14]:
# F1 score on Maria small/clean test dataset, model as-is.
from sklearn import metrics

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary', shuffle=False)

predictions = model3.predict_generator(test_generator, steps=len(test_generator))

val_targ = []
for i in range(0,len(test_generator)):
    val_targ.extend(np.array(test_generator[i][1]))

val_predict = (np.asarray(predictions)).round().flatten()

print(val_targ[50:100])
print(val_predict[50:80])

val_f1 = metrics.f1_score(val_targ, val_predict)
val_recall = metrics.recall_score(val_targ, val_predict)
val_precision = metrics.precision_score(val_targ, val_predict)
val_acc = metrics.accuracy_score(val_targ, val_predict)

print("F1: {0}, Recall: {1}, Precision: {2}, Accuracy: {3}".format(val_f1, val_recall, val_precision, val_acc))

Found 384 images belonging to 2 classes.
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0.
 1. 1. 1. 1. 1. 0.]
F1: 0.8414376321353065, Recall: 0.8844444444444445, Precision: 0.8024193548387096, Accuracy: 0.8046875


In [2]:
model4 = load_model('1024_512_dl_2dropout_1conv_200ep_7030.h5')

In [3]:
model4.summary()

test_dir = '/home/apando/datasets/building_crops_noaa_rezoom_des/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary')
test_results = model4.evaluate_generator(test_generator, steps=len(test_generator))
test_results

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
__________

[0.4222806865970294, 0.8359375]

In [4]:
# F1 score on Maria small/clean test dataset.
from sklearn import metrics

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary', shuffle=False)

predictions = model4.predict_generator(test_generator, steps=len(test_generator))

val_targ = []
for i in range(0,len(test_generator)):
    val_targ.extend(np.array(test_generator[i][1]))

val_predict = (np.asarray(predictions)).round().flatten()

print(val_targ[50:100])
print(val_predict[50:80])

val_f1 = metrics.f1_score(val_targ, val_predict)
val_recall = metrics.recall_score(val_targ, val_predict)
val_precision = metrics.precision_score(val_targ, val_predict)
val_acc = metrics.accuracy_score(val_targ, val_predict)

print("F1: {0}, Recall: {1}, Precision: {2}, Accuracy: {3}".format(val_f1, val_recall, val_precision, val_acc))

Found 384 images belonging to 2 classes.
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 1. 1. 0. 1. 1. 0.]
F1: 0.8668076109936577, Recall: 0.9111111111111111, Precision: 0.8266129032258065, Accuracy: 0.8359375


In [10]:
base_model = load_model('/home/apando/models/tomnod_everything_relu_Adam.h5')

In [11]:
base_model.summary()

test_dir = '/home/apando/datasets/building_crops_noaa_rezoom_des/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary')
test_results = base_model.evaluate_generator(test_generator, steps=len(test_generator))
test_results

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
__________

[1.1044417868057887, 0.6145833333333334]

In [15]:
# F1 score on Maria small/clean test dataset, model as-is.
from sklearn import metrics

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary', shuffle=False)

predictions = base_model.predict_generator(test_generator, steps=len(test_generator))

val_targ = []
for i in range(0,len(test_generator)):
    val_targ.extend(np.array(test_generator[i][1]))

val_predict = (np.asarray(predictions)).round().flatten()

print(val_targ[50:100])
print(val_predict[50:80])

val_f1 = metrics.f1_score(val_targ, val_predict)
val_recall = metrics.recall_score(val_targ, val_predict)
val_precision = metrics.precision_score(val_targ, val_predict)
val_acc = metrics.accuracy_score(val_targ, val_predict)

print("F1: {0}, Recall: {1}, Precision: {2}, Accuracy: {3}".format(val_f1, val_recall, val_precision, val_acc))

Found 384 images belonging to 2 classes.
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 0.]
F1: 0.6390243902439025, Recall: 0.5822222222222222, Precision: 0.7081081081081081, Accuracy: 0.6145833333333334


In [2]:
model5 = load_model('1024_512_dl_2dropout_1conv_400ep_7030.h5')

In [3]:
model5.summary()

test_dir = '/home/apando/datasets/building_crops_noaa_rezoom_des/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary')
test_results = model5.evaluate_generator(test_generator, steps=len(test_generator))
test_results

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
__________

[0.40088716087241966, 0.8411458333333334]

In [5]:
# F1 score on Maria small/clean test dataset.
from sklearn import metrics

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary', shuffle=False)

predictions = model5.predict_generator(test_generator, steps=len(test_generator))

val_targ = []
for i in range(0,len(test_generator)):
    val_targ.extend(np.array(test_generator[i][1]))

val_predict = (np.asarray(predictions)).round().flatten()

print(val_targ[50:100])
print(val_predict[50:80])

val_f1 = metrics.f1_score(val_targ, val_predict)
val_recall = metrics.recall_score(val_targ, val_predict)
val_precision = metrics.precision_score(val_targ, val_predict)
val_acc = metrics.accuracy_score(val_targ, val_predict)

print("F1: {0}, Recall: {1}, Precision: {2}, Accuracy: {3}".format(val_f1, val_recall, val_precision, val_acc))

Found 384 images belonging to 2 classes.
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 1. 1. 0.]
F1: 0.8682505399568035, Recall: 0.8933333333333333, Precision: 0.8445378151260504, Accuracy: 0.8411458333333334


In [6]:
model6 = load_model('/home/apando/models/tomnod_everything_relu_Adam-frozen_base_data_aug.h5')

In [7]:
model6.summary()

test_dir = '/home/apando/datasets/building_crops_noaa_rezoom_des/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary')
test_results = model6.evaluate_generator(test_generator, steps=len(test_generator))
print(test_results)

# F1 score on Maria small/clean test dataset.
from sklearn import metrics

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary', shuffle=False)

predictions = model6.predict_generator(test_generator, steps=len(test_generator))

val_targ = []
for i in range(0,len(test_generator)):
    val_targ.extend(np.array(test_generator[i][1]))

val_predict = (np.asarray(predictions)).round().flatten()

print(val_targ[50:100])
print(val_predict[50:80])

val_f1 = metrics.f1_score(val_targ, val_predict)
val_recall = metrics.recall_score(val_targ, val_predict)
val_precision = metrics.precision_score(val_targ, val_predict)
val_acc = metrics.accuracy_score(val_targ, val_predict)

print("F1: {0}, Recall: {1}, Precision: {2}, Accuracy: {3}".format(val_f1, val_recall, val_precision, val_acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
__________

In [8]:
model7 = load_model('/home/apando/models/tomnod_everything_relu_Adam-frozen_base_no_aug.h5')

In [9]:
model7.summary()

test_dir = '/home/apando/datasets/building_crops_noaa_rezoom_des/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary')
test_results = model7.evaluate_generator(test_generator, steps=len(test_generator))
print(test_results)

# F1 score on Maria small/clean test dataset.
from sklearn import metrics

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary', shuffle=False)

predictions = model7.predict_generator(test_generator, steps=len(test_generator))

val_targ = []
for i in range(0,len(test_generator)):
    val_targ.extend(np.array(test_generator[i][1]))

val_predict = (np.asarray(predictions)).round().flatten()

print(val_targ[50:100])
print(val_predict[50:80])

val_f1 = metrics.f1_score(val_targ, val_predict)
val_recall = metrics.recall_score(val_targ, val_predict)
val_precision = metrics.precision_score(val_targ, val_predict)
val_acc = metrics.accuracy_score(val_targ, val_predict)

print("F1: {0}, Recall: {1}, Precision: {2}, Accuracy: {3}".format(val_f1, val_recall, val_precision, val_acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
__________

In [10]:
model8 = load_model('/home/apando/models/tomnod_everything_relu_Adam-frozen_base_1024_fc.h5')

model8.summary()

test_dir = '/home/apando/datasets/building_crops_noaa_rezoom_des/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary')
test_results = model8.evaluate_generator(test_generator, steps=len(test_generator))
print(test_results)

# F1 score on Maria small/clean test dataset.
from sklearn import metrics

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary', shuffle=False)

predictions = model8.predict_generator(test_generator, steps=len(test_generator))

val_targ = []
for i in range(0,len(test_generator)):
    val_targ.extend(np.array(test_generator[i][1]))

val_predict = (np.asarray(predictions)).round().flatten()

print(val_targ[50:100])
print(val_predict[50:80])

val_f1 = metrics.f1_score(val_targ, val_predict)
val_recall = metrics.recall_score(val_targ, val_predict)
val_precision = metrics.precision_score(val_targ, val_predict)
val_acc = metrics.accuracy_score(val_targ, val_predict)

print("F1: {0}, Recall: {1}, Precision: {2}, Accuracy: {3}".format(val_f1, val_recall, val_precision, val_acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
__________

In [11]:
model9 = load_model('1024_512_dl_2dropout_2conv_100ep_7030.h5')

model9.summary()

test_dir = '/home/apando/datasets/building_crops_noaa_rezoom_des/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary')
test_results = model9.evaluate_generator(test_generator, steps=len(test_generator))
print(test_results)

# F1 score on Maria small/clean test dataset.
from sklearn import metrics

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
                    test_dir, 
                    target_size = (150,150),
                    batch_size = 16,
                    class_mode = 'binary', shuffle=False)

predictions = model9.predict_generator(test_generator, steps=len(test_generator))

val_targ = []
for i in range(0,len(test_generator)):
    val_targ.extend(np.array(test_generator[i][1]))

val_predict = (np.asarray(predictions)).round().flatten()

print(val_targ[50:100])
print(val_predict[50:80])

val_f1 = metrics.f1_score(val_targ, val_predict)
val_recall = metrics.recall_score(val_targ, val_predict)
val_precision = metrics.precision_score(val_targ, val_predict)
val_acc = metrics.accuracy_score(val_targ, val_predict)

print("F1: {0}, Recall: {1}, Precision: {2}, Accuracy: {3}".format(val_f1, val_recall, val_precision, val_acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
__________